#Pipeline Classes - AI Democracy

This notebook is to implement the classes responsible for the preprocessing pipeline, note that it assumes that you already have passed the text to the TypoParser functions in previous notebooks. All these objects are design to work only with the text (X) and label (y) columns.

# Importing main librarie

In [ ]:
!pip install -U spacy

     |████████████████████████████████| 12.8MB 290kB/s 
     |████████████████████████████████| 624kB 41.3MB/s 
     |████████████████████████████████| 460kB 26.0MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 9.1MB 62.2MB/s 
     |████████████████████████████████| 122kB 53.0MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107107 sha256=ce4fdbbe5bf83128faf9d479ff503ce3c3560f805ed5580dcc80032581596e77
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found e

In [ ]:
!python -m spacy download pt_core_news_lg

2021-06-14 02:41:24.496983: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 578.1MB 27kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [ ]:
import pandas as pd
import re
import nltk
import numpy as np
import pickle
import spacy
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec, KeyedVectors

# Loading the Data

In [ ]:
with open('no_typos (4).pkl', 'rb') as f:
    df = pickle.load(f)

X = df['Text']

#I don't have the classification yet, so
#y = df['classification'] #Binary 0 - interrupt; 1 - continuity

# Tokenizer

In [ ]:
class CustomTokenizer():
    
    def __init__(self, mappers='', custom_specials='default'):
        if custom_specials == 'default':
            self.custom_specials = "!\"#$%&'()*+,¸./:;<=>?@[\]^_`{|}-–⎯—«»´°‘’…~ªº€0123456789"
        else:
            self.custom_specials = custom_specials
        
        if mappers:
            with open(mappers, 'rb') as f:
                mappers = pickle.load(f)
                self.person_map = mappers[0]
                self.party_map = mappers[1]
    
    #Find the Names and Parties of politicians and make them as a unique token
    #Ex.: "Inês de Sousa Real" --> ["Inês de Sousa Real"], not ["Inês", "de", "Sousa", "Real"]
    def fit(self, dataframe):
        person_mapper = {}
        for person in pd.Series(dataframe['Person'].unique()).to_list():
            person_mapper[''.join(person.lower().split())] = person
        
        party_mapper = {}
        for party in pd.Series(dataframe['Party'].unique()).to_list():
            party = str(party)
            party_mapper[''.join(party.lower().split())] = party
        
        mappers = (person_mapper, party_mapper)

        self.person_map = person_mapper
        self.party_map = party_mapper

        with open('mappers.pkl', 'wb') as f:
            pickle.dump(mappers, f)

    def remove_specials_chars(self, text):
        for special_char in self.custom_specials:
            text = text.replace(special_char, ' ')
        text = text.replace('CDS PP', 'CDS-PP')
        return text
    
    #Apply the mapper, so a name becomes a single concatenated lowered string
    #ex.: "Inês de Sousa Real" --> "inesdesousareal"
    def apply_mappers(self, text):

        for person in self.person_map:
             text = text.replace(self.person_map[person], person)

        for party in self.party_map:
            party = str(party)
            text = text.replace(self.party_map[party], party)
        
        return text

    def convert_text(self, text):
        #converts to lowercase and split the words
        text = text.lower()
        words = text.split()
        
        return words
    
    def transform(self, X):
        X = X.apply(self.remove_specials_chars)
        X = X.apply(self.apply_mappers)
        X = X.apply(self.convert_text)
        return X


In [ ]:
tokenizer = CustomTokenizer()
tokenizer.fit(df)
X = tokenizer.transform(X)

In [ ]:
X[0]

['dirijo',
 'um',
 'abraço',
 'a',
 'todos',
 'neste',
 'regresso',
 'dos',
 'plenários',
 'à',
 'casa',
 'da',
 'democracia',
 'esperávamos',
 'que',
 'nesta',
 'altura',
 'já',
 'pudéssemos',
 'ter',
 'regras',
 'mais',
 'flexíveis',
 'mas',
 'infelizmente',
 'os',
 'números',
 'e',
 'as',
 'consequências',
 'concretas',
 'não',
 'nos',
 'permitem',
 'tal',
 'e',
 'portanto',
 'continuamos',
 'no',
 'essencial',
 'com',
 'as',
 'regras',
 'que',
 'presidiram',
 'aos',
 'últimos',
 'plenários',
 'da',
 'sessão',
 'legislativa',
 'srs',
 'deputados',
 'da',
 'nossa',
 'ordem',
 'do',
 'dia',
 'constam',
 'declarações',
 'políticas',
 'porém',
 'antes',
 'disso',
 'a',
 'sr',
 'secretária',
 'mariadaluzrosinha',
 'fará',
 'o',
 'favor',
 'de',
 'anunciar',
 'a',
 'entrada',
 'de',
 'algumas',
 'iniciativas',
 'tem',
 'a',
 'palavra',
 'sr',
 'secretária']

# Stopwords

In [ ]:
class StopwordsParser():

    def __init__(self, stopwords_file=''):
        self.stopwords = open(stopwords_file, 'r').read().splitlines()
        
    def fit(self):
        pass

    def remove_stopwords(self, text):
        text = [token for token in text if token not in self.stopwords]
        return text

    def transform(self, X):
        X = X.apply(self.remove_stopwords)
        return X


In [ ]:
stopwords_parser = StopwordsParser('complete_stopwords_set.txt')
X = stopwords_parser.transform(X)

In [ ]:
X[0]

['dirijo',
 'abraço',
 'todos',
 'neste',
 'regresso',
 'plenários',
 'casa',
 'democracia',
 'esperávamos',
 'nesta',
 'altura',
 'pudéssemos',
 'ter',
 'regras',
 'flexíveis',
 'infelizmente',
 'números',
 'consequências',
 'concretas',
 'permitem',
 'tal',
 'portanto',
 'continuamos',
 'essencial',
 'regras',
 'últimos',
 'plenários',
 'sessão',
 'legislativa',
 'ordem',
 'dia',
 'constam',
 'declarações',
 'políticas',
 'porém',
 'antes',
 'disso',
 'secretária',
 'mariadaluzrosinha',
 'fará',
 'favor',
 'anunciar',
 'entrada',
 'algumas',
 'iniciativas',
 'palavra',
 'secretária']

# Lemmatizer

In [ ]:
class CustomLemmatizer():
    def __init__(self, mappers=''):
        self.nlp = spacy.load('pt_core_news_lg',
                              exclude=['attribute_ruler', 'tok2vec', 'morphologizer',
                                       'parser', 'senter', 'ner', 'attribute_ruler'])
        self.nlp.max_length = 6136000

        if mappers:
            with open(mappers, 'rb') as f:
                mappers = pickle.load(f)
                self.person_map = mappers[0]
                self.party_map = mappers[1]

    def fit(self):
        pass
    
    def undo_mapping(self, tokens):
        #Deixando nomes de pessoas como tokens legiveis novamente
        for i, word in enumerate(tokens):
            if word in self.person_map:
                tokens[i] = self.person_map[word]
            elif word in self.party_map:
                tokens[i] = self.party_map[word]
        return tokens
    
    def normalize_tokens(self, tokens):
        meaningful_string = ' '.join(tokens)
        spacy_object = self.nlp(meaningful_string)
        normalized_tokens = [token.lemma_ for token in spacy_object]
        return normalized_tokens

    def transform(self, X):
        X = X.apply(self.normalize_tokens)
        X = X.apply(self.undo_mapping)
        return X

In [ ]:
lemmatizer = CustomLemmatizer('mappers.pkl')
X = lemmatizer.transform(X)

In [ ]:
X[0]

['dirigir',
 'abraçar',
 'todo',
 'neste',
 'regressar',
 'plenário',
 'casar',
 'democracia',
 'esperar',
 'nesta',
 'altura',
 'poder',
 'ter',
 'regrar',
 'flexível',
 'infelizmente',
 'número',
 'consequência',
 'concreto',
 'permitir',
 'tal',
 'portanto',
 'continuar',
 'essencial',
 'regrar',
 'último',
 'plenário',
 'sessão',
 'legislativo',
 'ordem',
 'dia',
 'constar',
 'declaração',
 'político',
 'porém',
 'antar',
 'disso',
 'Secretário',
 'Maria da Luz Rosinha',
 'fazer',
 'favor',
 'anunciar',
 'entrar',
 'algum',
 'iniciativo',
 'palavra',
 'Secretário']

#Word2Vec

In [1]:
import pandas as pd
import numpy as np
import pickle
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec, KeyedVectors

In [2]:
df = pd.read_excel('with_clean_texts.xlsx')
df.replace('nan', np.nan, inplace=True)
X = df['Normalized_Tokens']

df['Text'] = X
data_mask = ~pd.isna(df['Y_true'])
data = df[data_mask].copy()
data = data.reset_index(drop=True)

X_new = data['Text'].to_list()

In [3]:
sentences_ = X
w2v_10 = Word2Vec(sentences=sentences_, size=10, min_count=1, sg=1)
w2v_50 = Word2Vec(sentences=sentences_, size=50, min_count=1, sg=1)
w2v_100 = Word2Vec(sentences=sentences_, size=100, min_count=1, sg=1)
w2v_300 = Word2Vec(sentences=sentences_, size=300, min_count=1, sg=1)

In [4]:
X_10 = []
X_50 = []
X_100 = []
X_300 = []

for sentence in X_new:
    act_sent_10 = []
    act_sent_50 = []
    act_sent_100 = []
    act_sent_300 = []
    for word in sentence:
        act_sent_10.append(w2v_10.wv[word])
        act_sent_50.append(w2v_50.wv[word])
        act_sent_100.append(w2v_100.wv[word])
        act_sent_300.append(w2v_300.wv[word])
    if len(act_sent_10) > 0:
        stack_10 = np.stack(act_sent_10)
        X_10.append(np.median(stack_10, axis=0))
    else:
        X_10.append(np.zeros(10))
    if len(act_sent_50) > 0:
        stack_50 = np.stack(act_sent_50)
        X_50.append(np.median(stack_50, axis=0))
    else:
        X_50.append(np.zeros(50))
    if len(act_sent_100) > 0:
        stack_100 = np.stack(act_sent_100)
        X_100.append(np.median(stack_100, axis=0))
    else:
        X_100.append(np.zeros(100))
    if len(act_sent_300) > 0:
        stack_300 = np.stack(act_sent_300)
        X_300.append(np.median(stack_300, axis=0))
    else:
        X_300.append(np.zeros(300))

In [5]:
with open('X_10.pkl', 'wb') as f:
        pickle.dump(X_10, f)
with open('X_50.pkl', 'wb') as f:
        pickle.dump(X_50, f)
with open('X_100.pkl', 'wb') as f:
        pickle.dump(X_100, f)
with open('X_300.pkl', 'wb') as f:
        pickle.dump(X_300, f)

# Embeddings:

In [6]:
class CustomEmbeddings():
    def __init__(self, model='', vector_size=100, window_size=2):
        if model:
            self.model = KeyedVectors.load_word2vec_format(model)
        self.vector_size = vector_size
        self.window_size = window_size

    #If no model was given, then apply doc2vec as default
    def fit(self, X):
        documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(X)]
        self.model = Doc2Vec(
            documents=documents,
            vector_size=self.vector_size,
            window=self.window_size,
            min_count=1
        )

    def save_model(self, document_name):
        self.model.save(document_name)

    def transform(self, X):
        X = X.apply(self.model.infer_vector)
        return X

In [7]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(X)]

d2v_10 = Doc2Vec(documents=documents, 
                  vector_size=10,
                  min_count=1)
d2v_50 = Doc2Vec(documents=documents, 
                  vector_size=50,
                  min_count=1)
d2v_100 = Doc2Vec(documents=documents, 
                  vector_size=100,
                  min_count=1)
d2v_300 = Doc2Vec(documents=documents, 
                  vector_size=300,
                  min_count=1)

In [8]:
X_d2v_10 = np.stack(pd.Series(X_new).apply(d2v_10.infer_vector).to_numpy())
X_d2v_50 = np.stack(pd.Series(X_new).apply(d2v_50.infer_vector).to_numpy())
X_d2v_100 = np.stack(pd.Series(X_new).apply(d2v_100.infer_vector).to_numpy())
X_d2v_300 = np.stack(pd.Series(X_new).apply(d2v_300.infer_vector).to_numpy())

In [9]:
with open('X_d2v_10.pkl', 'wb') as f:
        pickle.dump(X_d2v_10, f)
with open('X_d2v_50.pkl', 'wb') as f:
        pickle.dump(X_d2v_50, f)
with open('X_d2v_100.pkl', 'wb') as f:
        pickle.dump(X_d2v_100, f)
with open('X_d2v_300.pkl', 'wb') as f:
        pickle.dump(X_d2v_300, f)

In [11]:
X_new[0]

"['obrigar', 'Secretária', 'Maria', 'da', 'Luz', 'Rosinha', 'pois', 'condição', 'iniciar', 'primeiro', 'pontar', 'agendar', 'consistir', 'declaração', 'político', 'primeiro', 'declaração', 'político', 'caber', 'grupar', 'parlamentar', 'PS', 'antar', 'dar', 'palavra', 'Luís', 'Moreira', 'Testa', 'relembrar', 'regrar', 'funcionamento', 'plenário', 'continuar', 'mesmo', 'vigor', 'antar', 'féria', 'infelizmente', 'realidade', 'fazer', 'favor']"

In [12]:
d2v_10.infer_vector(X_new[0])

array([-0.44160825, -0.04035217, -0.18447252,  0.2545964 ,  0.19554137,
        0.28900757,  0.13697784, -0.03406145, -0.2350337 ,  0.05136514],
      dtype=float32)

In [22]:
d2v_10.wv[X_new[0]]

KeyError: ignored